## Eliminasi Gaussian

Pada bagian ini kita mendefinisikan beberapa fungsi Python untuk membantu kita menyelesaikan sistem linier. Algoritme ini dikenal sebagai Eliminasi Gaussian, yang nanti kita akan sebut dengan  **eliminasi**. Ide eliminasi adalah menukar sistem yang ada dengan sistem lain yang memiliki penyelesaian yang sama, namun lebih mudah diselesaikan. Untuk mencapai tujuan ini, kami akan melakukan serangkaian langkah yang disebut **operasi baris** yang mempertahankan penyelesaian sistem sekaligus secara bertahap membuat penyelesaian lebih mudah dilakukan. Ada tiga operasi yang dapat kami lakukan.
1. Tukarkan posisi kedua persamaan.
2. Kalikan persamaan dengan bilangan apa pun yang bukan nol.
3. Gantikan suatu persamaan dengan jumlah persamaan itu sendiri dan kelipatan persamaan lainnya.



<a id='GE1'></a>
### Contoh 1:  Operasi baris dan eliminasi

Perhatikan contoh berikut.

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 + x_2 + 8x_3 & = & 18\\
4x_1 + 2x_2 -3x_3 & = & -2
\end{eqnarray*}
$$

Kita dapat menukar persamaan pertama dengan yang persamaan terakhir

$$
\begin{eqnarray*}
4x_1 + 2x_2 -3x_3 & = & -2 \\
2x_1 + x_2 + 8x_3 & = & 18\\
x_1 - x_2 + x_3 & = & 3
\end{eqnarray*}
$$

atau kita dapat mengalikan persamaan pertama dengan$5$,

$$
\begin{eqnarray*}
5x_1 - 5x_2 + 5x_3 & = & 15\\
2x_1 + x_2 + 8x_3 & = & 18\\
4x_1 + 2x_2 -3x_3 & = & -2
\end{eqnarray*}
$$

atau kita dapat menambahkan 2 kali persamaan pertama dengan persamaan terakhir

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 + x_2 + 3x_3 & = & 8\\
6x_1 \quad\quad -x_3 & = & 4
\end{eqnarray*}
$$

Operasi terakhir adalah yang paling penting karena memungkinkan kita *mengeliminasi* variabel dari salah satu persamaan. Perhatikan bahwa persamaan ketiga tidak lagi mengandung suku $x_2$. Ini adalah kunci dari algoritma eliminasi.

Untuk tujuan komputasi, kita dapat membuang nama variabel dan simbol "=", dan mengatur semua angka sebenarnya dalam sebuah array.

$$
\begin{equation}
\left[ \begin{array}{rrrr} 1 & -1 & 1 & 3 \\ 2 & 1 & 8 & 18 \\ 4 & 2 & -3 & -2 \end{array}\right]
\end{equation}
$$

Sekarang mari kita buat array NumPy dengan nilai-nilai ini. Kita akan memberi nama pada array $\texttt{A}$, sehingga kita bisa merujuknya nanti.

In [ ]:
import numpy as np
A=np.array([[1,-1,1,3],[2,1,8,18],[4,2,-3,-2]])

Kita bisa mulai melakukan operasi pada array, namun pertama kali kita akan menulis beberapa kode program python yang akan digunakan untuk  perhitungan selanjutnya.

In [ ]:
def RowSwap(A,k,l):
# =============================================================================
#     A is a NumPy array.  RowSwap will return duplicate array with rows
#     k and l swapped.
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A

    B = np.copy(A).astype('float64')

    for j in range(n):
        temp = B[k][j]
        B[k][j] = B[l][j]
        B[l][j] = temp

    return B

def RowScale(A,k,scale):
# =============================================================================
#     A is a NumPy array.  RowScale will return duplicate array with the
#     entries of row k multiplied by scale.
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A

    B = np.copy(A).astype('float64')

    for j in range(n):
        B[k][j] *= scale

    return B

def RowAdd(A,k,l,scale):
# =============================================================================
#     A is a numpy array.  RowAdd will return duplicate array with row
#     l modifed.  The new values will be the old values of row l added to
#     the values of row k, multiplied by scale.
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A

    B = np.copy(A).astype('float64')

    for j in range(n):
        B[l][j] += B[k][j]*scale

    return B



Kami sekarang memiliki tiga fungsi  yang disebut $\texttt{RowSwap}$,$\texttt{RowScale}$, dan $\texttt{RowAdd}$. Mari kita mencobanya untuk melihat apa yang akan hasilkan oleh fungsi fungsi tersebut.

In [ ]:
B1 = RowSwap(A,0,2)
B2 = RowScale(A,2,0.5)
B3 = RowAdd(A,0,1,2)

In [ ]:
print(A)
print('\n')
print(B2)

[[ 1 -1  1  3]
 [ 2  1  8 18]
 [ 4  2 -3 -2]]


[[ 1.  -1.   1.   3. ]
 [ 2.   1.   8.  18. ]
 [ 2.   1.  -1.5 -1. ]]


Tujuan dari eliminasi adalah untuk melakukan operasi baris pada array ini untuk menghasilkan array baru dengan struktur seperti berikut.

$$
\begin{equation}
\left[ \begin{array}{cccc} 1 & * & * & * \\ 0 & 1 & * & * \\ 0 & 0 & 1 & * \end{array}\right]
\end{equation}
$$

*(Catatan simbol * disini menyatakan nilai nilai yang tidak diketahui yang mungkin 0 atau 1.)*

Kami akan menjalankan operasi baris dan menyimpan tahapan tersebut sebagai array  baru spada  setiap langkah. Misalnya, kita menyebutnya dengan  $\texttt{A1}$, $\texttt{A2}$, $\texttt{A3}$, dll. Dengan cara ini kita dapat memeriksa prosesnya, atau membuat perubahan sesuai yang kita inginkan

In [ ]:
## Tambahkan -2 kali baris 0 ke baris 1
A1 = RowAdd(A,0,1,-2)
print(A1,'\n')

## Tambahkan -4 kali baris 0 ke baris 2
A2 = RowAdd(A1,0,2,-4)
print(A2,'\n')

## Tambahkan -2 kali baris 1 ke baris 2
A3 = RowAdd(A2,1,2,-2)
print(A3,'\n')

## Kalikan baris 1 dengan 1/3
A4 = RowScale(A3,1,1.0/3)
print(A4,'\n')

## Kalikan baris 2 dengan 1/19
A5 = RowScale(A4,2,1.0/-19.)
print(A5)

[[ 1. -1.  1.  3.]
 [ 0.  3.  6. 12.]
 [ 4.  2. -3. -2.]] 

[[  1.  -1.   1.   3.]
 [  0.   3.   6.  12.]
 [  0.   6.  -7. -14.]] 

[[  1.  -1.   1.   3.]
 [  0.   3.   6.  12.]
 [  0.   0. -19. -38.]] 

[[  1.  -1.   1.   3.]
 [  0.   1.   2.   4.]
 [  0.   0. -19. -38.]] 

[[ 1. -1.  1.  3.]
 [ 0.  1.  2.  4.]
 [-0. -0.  1.  2.]]


Sekarang mari kita terjemahkan array matrik ini kembali ke sistem persamaan

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
x_2 + 2x_3 & = & 4\\
x_3 & = & 2
\end{eqnarray*}
$$

Setelah langkah-langkah eliminasi, kita mempunyai apa yang dikenal sebagai sistem **segitiga atas**. Penyelesain terhadap sistem ini mudah ditemukan,yaitu dengan menemukan penyelesaian dari persamaan terakhir. Persamaan terakhir memberitahu kita bahwa $x_3 = 2$. Jika kita mensubstitusikan nilai tersebut ke persamaan kedua, kita mendapatkan $x_2 = 0$. Terakhir, jika kita substitusikan kembali nilai-nilai $x_3$ dan $x_2$ ini ke persamaan pertama, kita akan mendapatkan $x_1 = 1.$ Proses mencari penyelesain sistem segitiga atas ini disebut **substitusi balik**.

### Contoh 2:  Eliminasi pada array acak

Jika suatu sistem persamaan mempunyai solusi, algoritma eliminasi akan selalu menghasilkan sistem segitiga atas yang dapat diselesaikan dengan substitusi balik. Dalam contoh berikutnya, kita melakukan perhitungan dengan sedikit perubahan agar dapat digunakan sebagai acuan untuk melakukan perhitungan secara umum. Kali ini  kita menggunakan fungsi $\texttt{RowAdd}$, untuk mengatur parameter *scale* berdasarkan nilai dalam array.  

Kita akan membuat matriks dengan nilai elemen-elemen acak menggunakan modul $\texttt{random}$.

In [ ]:
R = np.random.randint(-8,8,size=(3,4))
print(R)

[[-7 -7  2 -6]
 [-5 -4  4  4]
 [ 5  7  5 -6]]


In [ ]:
## Skalakan baris pertama berdasarkan elemen pertama pada baris tersebut.
R1 = RowScale(R,0,1.0/R[0][0])
## Tambahkan baris pertama ke baris kedua berdasarkan elemen pertama di baris kedua.
R2 = RowAdd(R1,0,1,-R[1][0])
## Tambahkan baris pertama ke baris terakhir berdasarkan elemen pertama di baris terakhir.
R3 = RowAdd(R2,0,2,-R2[2][0])
## Skalakan baris kedua berdasarkan elemen kedua di baris tersebut.
R4 = RowScale(R3,1,1.0/R3[1][1])
## Tambahkan baris kedua ke baris terakhir berdasarkan elemen kedua di baris terakhir.
R5 = RowAdd(R4,1,2,-R4[2][1])
## Skalakan baris terakhir berdasarkan elemen terakhir pada baris tersebut.
R6 = RowScale(R5,2,1.0/R5[2][2])
print(R6)

[[  1.           1.          -0.28571429   0.85714286]
 [  0.           1.           2.57142857   8.28571429]
 [  0.           0.           1.         -20.88888889]]


Setelah kami memahami cara kerja operasi baris, dan kami yakin bahwa operasi tersebut berfungsi dengan benar, sehingga kita tidak perlu menyimpan setiap prose eliminasi. Oleh karena itu kita dapat menyederhanakan kode dengan menggunakan kembali nama array yang sama untuk setiap langkah. Kode berikut akan menghasilkan hasil yang sama.

```
R = RowScale(R,0,1.0/R[0][0])
R = RowAdd(R,0,1,-R[1][0])
R = RowAdd(R,0,2,-R[2][0])
R = RowScale(R,1,1.0/R[1][1])
R = RowAdd(R,1,2,-R[2][1])
R = RowScale(R,2,1.0/R[2][2])
print(R)
```


Jalankan kode dalam contoh ini beberapa kali. Setiap kali array $\texttt{R}$ dibuat, array tersebut akan diisi dengan bilangan bulat acak antara -8 dan 8. Apakah kode tersebut selalu menghasilkan sistem segitiga atas yang siap untuk disubstitusi balik? perhtaikan apkah Anda  mengetahui bagian mana dari proses yang mungkin gagal, dan mengapa?

### Contoh 3: Menentukan pivot

Seperti yang bisa kita lihat, kode pada contoh terakhir gagal jika angka nol muncul sebagai salah satu entri dalam larik yang kita bagi untuk mendapatkan faktor skala. Elemen penting ini disebut dengan **pivot**, dan lokasinya dalam matriks disebut **posisi pivot**. Menurut definisi, pivot harus bukan nol. Jika angka nol muncul di posisi pivot pada  tahap eliminasi, kita dapat  menukar urutan baris untuk memindahkan elemen bukan nol ke posisi pivot. Baiklah, kita coba untuk sistem persamaan  tertentu sebelum mencoba menulis kode program yang akan digunakan pada  sembarang sistem persamaan


$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 - 2x_2 + 4x_3 & = & 8\\
3x_1 \quad\quad -9x_3 & = & 0
\end{eqnarray*}
$$


In [ ]:
G=np.array([[1,-1,1,3],[2,-2,4,8],[3,0,-9,0]])
print(G)


[[ 1 -1  1  3]
 [ 2 -2  4  8]
 [ 3  0 -9  0]]


In [ ]:
## Tambahkan -2 kali baris 0 ke baris 1
G1 = RowAdd(G,0,1,-2)
## Tambahkan -3 kali baris 0 ke baris 2
G2 = RowAdd(G1,0,2,-3)
print(G2)

[[  1.  -1.   1.   3.]
 [  0.   0.   2.   2.]
 [  0.   3. -12.  -9.]]


Disini **ada angka nol** di posisi pivot tengah. Kita dapat menukar persamaan tengah dan persamaan terakhir untuk melanjutkan eliminasi.

In [ ]:
## Swap rows 1 and 2
G3 = RowSwap(G2,1,2)
## Scale the new row 1 by 1/3
G4 = RowScale(G3,1,1./3)
## Scale the new row 2 by 1/2
G5 = RowScale(G4,2,1./2)
print(G5)

[[ 1. -1.  1.  3.]
 [ 0.  1. -4. -3.]
 [ 0.  0.  1.  1.]]


Kami tulis sistem itu lagi sebagai serangkaian persamaan yang berubah susunannya seperti berikut.

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
x_2 - 4x_3 & = & -3\\
x_3 & = & 1
\end{eqnarray*}
$$

Gunakan subsitusi balik, kita menemukan$x_2 = 1$ dan $x_1=3$.

Perlu dicatat bahwa pertukaran baris hanya diperlukan sebagai masalah penyusunan kembali persamaan. Berikut adalah cara lain yang bisa kita lakukan pada  sistem yang persamaan seperti diatas


In [ ]:
## Skalakan bari 1 dengan 1/2
G3_alternative = RowScale(G2,1,1./2)
## Skalakan baris 2 dengan 1/3
G4_alternative = RowScale(G3_alternative,2,1./3)
print(G4_alternative)

[[ 1. -1.  1.  3.]
 [ 0.  0.  1.  1.]
 [ 0.  1. -4. -3.]]


Persamaan yang  yang dihasilkan  adalah sistem persamaan yang sama, tetapi urutan persamaan berbeda.

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
x_3 & = & 1 \\
x_2 - 4x_3 & = & -3
\end{eqnarray*}
$$

Ide substitusi balik juga dapat diterapkan pada sistem ini, namun pengaturan algoritmanya sedikit lebih rumit.

<a id='GE4'></a>
### Contoh 4:  Eliminasi yang gagal

Mari kita sedikit lakukan perubahan  pada sistem pada contoh sebelumnya dan amati contoh ini bagaimana proses eliminasi dapat terhenti.

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 - 2x_2 + 4x_3 & = & 8\\
3x_1 -3x_2 -9x_3 & = & 0
\end{eqnarray*}
$$


In [ ]:
H = np.array([[1,-1,1,3],[2,-2,4,8],[3,-3,-9,0]])
print(H)

[[ 1 -1  1  3]
 [ 2 -2  4  8]
 [ 3 -3 -9  0]]


In [ ]:
## Tambahkan -2 kali baris 0 ke baris 1
H1 = RowAdd(H,0,1,-2)
## Tambahkan -3 kali baris 0 ke baris 2
H2 = RowAdd(H1,0,2,-3)
print(H2)

[[  1.  -1.   1.   3.]
 [  0.   0.   2.   2.]
 [  0.   0. -12.  -9.]]


Dalam hal ini menukar baris kedua dan ketiga tidak dapat membantu peyelesain karena kedua baris memiliki nilai nol di elemen tengah. Artinya tidak ada pivot pada kolom kedua. Mari kita skalakan barisnya dan lihat hasilnya..

In [ ]:
## Kalikan baris 1 dengan 1/2
H3 = RowScale(H2,1,1./2)
## Kalikan baris 2 dengan -1/12
H4 = RowScale(H3,2,-1./12)
print(H4)

[[ 1.   -1.    1.    3.  ]
 [ 0.    0.    1.    1.  ]
 [-0.   -0.    1.    0.75]]


Pada persamaan tersebut, kita melihat bahwa sistem ini **tidak konsisten**. Dua persamaan terakhir saling bertentangan.

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
x_3 & = & 1 \\
x_3 & = & 0.75
\end{eqnarray*}
$$

Perhatikan bahwa kita tidak melakukan kesalahan apa pun dalam proses perhitungan dan tidak ada cara untuk menyusun ulang persamaan tersebut untuk mengatasi masalah tersebut. Oleh karena itu,sistem ini tidak mempunyai penyelesaian. Kami akan mambahas  ini kembali di bagian selanjutnya dan memberikan penjelasan  untuk sistem tersebut.



### Contoh 5:  Penyelesaian tidak tunggal

Dalam contoh eliminasi terakhir ini, kita mengamati kemungkinan hasil lain dari proses tersebut.

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 - 2x_2 + 4x_3 & = & 8\\
3x_1 -3x_2 +3x_3 & = & 9
\end{eqnarray*}
$$


In [ ]:
F = np.array([[1,-1,1,3],[2,-2,4,8],[1,-1,3,5]])
print(F)

[[ 1 -1  1  3]
 [ 2 -2  4  8]
 [ 1 -1  3  5]]


In [ ]:
## Tambahkan -2 kali baris 0 ke baris 1
F1 = RowAdd(F,0,1,-2)
## Tambahkan -3 kali baris 0 ke baris 2
F2 = RowAdd(F1,0,2,-1)
print(F2)

[[ 1. -1.  1.  3.]
 [ 0.  0.  2.  2.]
 [ 0.  0.  2.  2.]]


Mirip dengan contoh sebelumnya, kita melihat bahwa tidak ada cara untuk memasukkan nilai bukan nol ke kolom kedua pada baris kedua. Namun dalam kasus ini kita melihat bahwa persamaan kedua dan ketiga tidak saling bertentangan, namun sebenarnya merupakan persamaan yang sama.

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
2x_3 & = & 2\\
2x_3 & = & 2
\end{eqnarray*}
$$

Mari kita lakukan lagi dua operasi baris  untuk menyederhanakan sistem ini lebih jauh.

In [ ]:
## Tambahkan -1 kali baris 1 ke baris 2
F3 = RowAdd(F2,1,2,-1)
## Kalikan baris 1 dengan 1/2
F4 = RowScale(F3,1,0.5)
print(F4)

[[ 1. -1.  1.  3.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  0.  0.]]


Kita melihat dari  persamaan terakhir berlaku untuk semua nilai $x_1$, $x_2$, $x_3$, oleh karena itu ebenarnya hanya ada dua persamaan yang  menentukan nilai dari tiga nilai yang tidak diketahui.

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
x_3 & = & 1\\
0 & = & 0
\end{eqnarray*}
$$

Karena persamaan yang ditengah menunjukkan bahwa $x_3=1$, kita dapat memasukkan nilai tersebut ke persamaan pertama menggunakan ide substitusi balik. Hasilnya adalah persamaan tunggal $x_1-x_2 = 2$. Persamaan ini, dan sistem secara keseluruhan, memiliki *jumlah solusi tak terhingga*. Kami akan membaha lagi  dan akan memberikan rincian lebih lanjut tentang hal di bab berikutnya.

### Latihan

**Latihan 1:** Pada sistem di bawah ini, gunakan fungsi operasi baris untuk menghasilkan koefisien nol di lokasi koefisien 12. Lakukan ini dengan menghitung secara manual lalu buat array NumPy untuk menyatakan sistem persamaan dan gunakan fungsi operasi baris. (*Ada dua cara untuk membuat angka nol menggunakan $\texttt{RowAdd}$. Coba temukan keduanya.*)

$$
\begin{eqnarray*}
4x_1 - 2x_2 + 7x_3 & = & 2\\
x_1 + 3x_2 + 12x_3 & = & 4\\
-7x_1 \quad\quad - 3x_3 & = & -1
\end{eqnarray*}
$$


In [ ]:
## Koding disini

**Latihan 2:** Buat array NumPy untuk mewakili sistem persamaan di bawah ini. Tentukan koefisien mana yang harus sama dengan nol agar sistem tersebut berbentuk segitiga atas. Gunakan $\texttt{RowAdd}$ untuk menjalankan operasi baris dan kemudian cetak hasilnya.
  $$
\begin{eqnarray*}
3x_1 + 4x_2 \, - \,\,\,\,\,  x_3 &   =   & -6\\
-2x_2   +  10x_3  &   =   & -8\\
4x_2   \,  - \,\, 2x_3 &  =  & -2
\end{eqnarray*}
$$

In [ ]:
# Koding disini

**Latihan 3:** Lakukan proses eliminasi pada sistem berikut. Tentukan array NumPy dan manfaatkan fungsi operasi baris. Cetak hasil setiap langkah. Tuliskan sistem segitiga atas yang diwakili oleh array setelah menyelesaiakan semua langkah

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 + x_2 + 8x_3 & = & 18\\
4x_1 + 2x_2 -3x_3 & = & -2
\end{eqnarray*}
$$

In [ ]:
#Koding disini

**Latihan 4:** Gunakan operasi baris pada sistem di bawah ini untuk menghasilkan sistem **segitiga bawah**. Persamaan pertama sistem segitiga bawah hanya boleh berisi $x_1$ dan persamaan kedua hanya berisi $x_1$ dan $x_2$.

  $$
\begin{eqnarray*}
x_1 + 2x_2 + x_3 & = & 3\\
3x_1 - x_2 - 3x_3 & = & -1\\
2x_1 + 3x_2 + x_3 & = & 4
\end{eqnarray*}
$$

In [ ]:
#Koding disini

**Latihan 5:** Gunakan eliminasi untuk menentukan apakah sistem berikut **konsisten** atau **tidak konsisten**.

$$
\begin{eqnarray*}
x_1 - x_2 - x_3 & = & 4\\
2x_1 - 2x_2 - 2x_3 & = & 8\\
5x_1 - 5x_2 - 5x_3 & = & 20
\end{eqnarray*}
$$



In [ ]:
#Koding disini

**Latihan 6:** Gunakan eliminasi untuk menunjukkan bahwa sistem persamaan ini tidak mempunyai penyelesaian.

  $$
\begin{eqnarray*}
x_1  +  x_2 +  x_3 & = & 0\\
x_1 -  x_2 + 3x_3 & = & 3\\
-x_1 - x_2 - x_3 & = & 2
\end{eqnarray*}
$$
  


In [ ]:
#Koding disini

**Latihan 7:** Gunakan modul $\texttt{random}$ untuk menghasilkan array $3\times 4$ yang berisi bilangan bulat acak antara $0$ dan $5$. Tulis kode yang melakukan operasi baris untuk menghasilkan matriks dengan nilai nol pada baris pertama, kolom ketiga. Jalankan kode beberapa kali untuk memastikan kode berfungsi pada array acak **berbeda**. Apakah kodenya akan gagal?

In [ ]:
#Koding disini

**Latihan 8:** Mulai dari array yang mewakili sistem segitiga atas di **Contoh 1** ($\texttt{A5}$), gunakan operasi baris untuk menghasilkan array matrik dengan bentuk dibawah ini. Lakukan tahapan demi tahapan dan cetak lagi hasil Anda untuk memeriksa apakah Anda berhasil.

$$
\begin{equation}
\left[ \begin{array}{cccc} 1 & 0 & 0 & 1 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 2 \end{array}\right]
\end{equation}
$$

In [ ]:
#Koding disini

**Latihan 9:** Ulangi **Contoh 2** menggunakan modul $\texttt{random}$ untuk menghasilkan array $3\times 4$ yang terdiri dari bilangan riel acak, bukan bilangan bulat acak.

In [ ]:
#Koding disini.

**Latihan 10:** Tulis perulangan yang akan mengeksekusi kode eliminasi dalam **Contoh 2** pada 1000 array $3\times 4$ yang berbeda dari bilangan acak riel untuk melihat seberapa sering gagalnya.

In [ ]:
#Koding disini